In [1]:
# SKIP TO CHECKPOINT and only run from there (to use the precomputed .parquet file)
#   (otherwise it will take several minutes)

# The following is based on RI analysis from the following paper
# https://journals.ametsoc.org/view/journals/wefo/35/6/WAF-D-19-0253.1.xml#bib15
# This notebook has only done Atlantic RI)

# Note: as in the paper above, there is some overlap for rapid intensification events and forecasts
# an example would be: RI in 24h in one forecast, and then a subsequent forecast 12 hours later has the 12h with RI
# this results in some overlap (double counting)

import numpy as np
import pandas as pd
from datetime import datetime, timedelta

filepath_ti_errs_atl = '1989-present_OFCL_v_BCD5_ind_ATL_TI_errors.txt'
filepath_ti_errs_pac = '1989-present_OFCL_v_BCD5_ind_EPAC_TI_errors.txt'
filepath_ac_errs_atl = '1989-present_OFCL_v_BCD5_ind_ATL_AC_errors.txt'
filepath_ac_errs_pac = '1989-present_OFCL_v_BCD5_ind_EPAC_AC_errors.txt'

# only considering track intensity errors for the atlantic (filepath_ti_errs_atl) in this notebook

In [2]:
def read_intensity_err(filepath):
    header = {}
    with open(filepath, 'r') as f:
        # header lines
        ln = f.readline().strip()
        header['basin'] = ln.split(':')[1].strip()
        ln = f.readline().strip()
        header['models'] = ln.split(':')[1].strip().split(' ')
        ln = f.readline().strip()
        header['kt_range'] = ln.split(':')[1].strip()
        ln = f.readline().strip()
        header['subtropical'] = ln
        ln = f.readline().strip()
        header['extratropical'] = ln
        ln = f.readline().strip()
        header['dissipation'] = ln
        
        # only keep columns for official forecasts (drop baseline forecasts)
        if header['models'][0] == 'OFCL':
            drop_match_str1 = 'hI02'
            drop_match_str2 = 'hT02'
        else:
            drop_match_str1 = 'hI01'
            drop_match_str2 = 'hT01'

        # line space between header and data
        ln = f.readline().strip()
        
        # line with column names
        ln = f.readline().strip()
        column_names = list(filter(None,ln.split(' ')))
        
        drop_columns = []
        for column_name in column_names:
            if drop_match_str1 in column_name or drop_match_str2 in column_name:
                drop_columns.append(column_name)
        
        # loop over the data
        data = []
        while True:
            ln = f.readline()
            if not ln:
                break
            ln = ln.strip()
            row = list(filter(None,ln.split(' ')))
            data.append(row)

        # Create the pandas DataFrame
        df = pd.DataFrame(data, columns = column_names)
        # convert the time column to panda timestamp
        df['Date/Time'] = pd.to_datetime(df['Date/Time'], dayfirst=True)
        # rename it to 'ds' to be shorter
        df.rename(columns={'Date/Time': 'ds'}, inplace=True)
        # drop columns from base line forecast (CLIPER5 or SHIFOR5) and keep OFCL
        df.drop(columns=drop_columns, inplace=True)
        for column_name in list(df.keys()):
            if column_name != 'ds' and column_name != 'STMID': 
                df[column_name] = df[column_name].astype(float)
        # replace missing values (-9999) with NaN
        df.replace(-9999.0, np.NaN, inplace=True)
        # convert all but 'ds' and 'STMID' from string to float now
        df.reset_index(drop=True, inplace=True)
        return [header, df]

def is_rapid_intensification(valid_h, base_intensity, valid_intensity):
    ri = False
    if np.isnan(base_intensity) or np.isnan(valid_intensity):
        return ri
    if valid_h == 0:
        # don't do RI for base time (000h)
        return ri
    # https://journals.ametsoc.org/view/journals/wefo/35/6/WAF-D-19-0253.1.xml#bib15
    # 'RI is therefore defined as an increase of at least 20 kt in 12 h, 30 kt in 24 h, 45 kt in 36 h, and 55 kt in 48 h'
    intensity_change = valid_intensity - base_intensity
    rapid_intensification_threshold = np.NaN
    if valid_h <= 12:
        rapid_intensification_threshold = 20
    elif valid_h <= 24:
        rapid_intensification_threshold = 30
    elif valid_h <= 36:
        rapid_intensification_threshold = 45
    elif valid_h <= 48:
        rapid_intensification_threshold = 55
    # only consider rapid intensification for the above periods
    if np.isnan(rapid_intensification_threshold):
        return ri
    if (intensity_change >= rapid_intensification_threshold):
        ri = True
    return ri

In [3]:
[header, df] = read_intensity_err(filepath_ti_errs_atl)
# track and intensity (used for naming files)
error_category = 'TI'

In [4]:
# create mappings and create new columns for calculations and analysis

column_names = list(df.keys())
# examples of column names with 48 hours in parenthesis
# (48hI01) intensity valid forecast hours
intensity_err_column_name_to_hour = {}
intensity_err_hour_to_column_name = {}
# (48hFI) forecast intensity for valid hours of the forecast (missing so recalculate from data)
forecast_intensity_column_name_to_hour = {}
forecast_intensity_hour_to_column_name = {}
# (48hBI) best intensity for valid hours of the forecast
best_intensity_column_name_to_hour = {}
best_intensity_hour_to_column_name = {}
# (48hFRI) forecast rapid intensification
forecast_rapid_intensification_column_name_to_hour = {}
forecast_rapid_intensification_hour_to_column_name = {}
# (48hBRI) best intensity rapid intensification
best_intensity_rapid_intensification_column_name_to_hour = {}
best_intensity_rapid_intensification_hour_to_column_name = {}
# (48hDFI) changes in forecast intensity from base time to valid hour
forecast_intensity_change_column_name_to_hour = {}
forecast_intensity_change_hour_to_column_name = {}
# (48hDBI) changes in best intensity from base time to valid hour
best_intensity_change_column_name_to_hour = {}
best_intensity_change_hour_to_column_name = {}

for column_name in column_names:
    if 'hI' in column_name:
        # create mappings from intensity column names to hour
        h = int(column_name.split('hI')[0])
        intensity_err_column_name_to_hour[column_name] = h
        intensity_err_hour_to_column_name[h] = column_name
        
        # create extra columns for forecast intensity for each forecast hour
        forecast_intensity_column_name = f'{h}hFI'
        df[forecast_intensity_column_name] = np.NaN
        forecast_intensity_column_name_to_hour[forecast_intensity_column_name] = h
        forecast_intensity_hour_to_column_name[h] = forecast_intensity_column_name
        
        # create extra columns for best intensity for each forecast hour
        best_intensity_column_name = f'{h}hBI'
        df[best_intensity_column_name] = np.NaN
        best_intensity_column_name_to_hour[best_intensity_column_name] = h
        best_intensity_hour_to_column_name[h] = best_intensity_column_name

        # create extra columns for forecast RI categorization for each forecast hour
        forecast_rapid_intensification_column_name = f'{h}hFRI'
        df[forecast_rapid_intensification_column_name] = False
        forecast_rapid_intensification_column_name_to_hour[forecast_rapid_intensification_column_name] = h
        forecast_rapid_intensification_hour_to_column_name[h] = forecast_rapid_intensification_column_name
        
        # create extra columns for best intensity RI categorization for each forecast hour
        best_intensity_rapid_intensification_column_name = f'{h}hBRI'
        df[best_intensity_rapid_intensification_column_name] = False
        best_intensity_rapid_intensification_column_name_to_hour[best_intensity_rapid_intensification_column_name] = h
        best_intensity_rapid_intensification_hour_to_column_name[h] = best_intensity_rapid_intensification_column_name
        
        # create extra columns for changes in forecast intensity for each forecast hour
        # (forecast intensity at valid hour - forecast intensity at base hour)
        forecast_intensity_change_column_name = f'{h}hDFI'
        df[forecast_intensity_change_column_name] = np.NaN
        forecast_intensity_change_column_name_to_hour[forecast_intensity_change_column_name] = h
        forecast_intensity_change_hour_to_column_name[h] = forecast_intensity_change_column_name
        
        # create extra columns for changes in (best) intensity for each forecast hour
        # (best intensity at valid hour (from other row) - best intensity at base hour)
        best_intensity_change_column_name = f'{h}hDBI'
        df[best_intensity_change_column_name] = np.NaN
        best_intensity_change_column_name_to_hour[best_intensity_change_column_name] = h
        best_intensity_change_hour_to_column_name[h] = best_intensity_change_column_name

# column for if any of the forecast hours show rapid intensification
# (forecast) rapid intensification
df['FRI'] = False
df['FRI_count'] = 0

# column for if any of the (best intensity) valid hours of a forecast show rapid intensification
# (best intensity) rapid intensification
df['BRI'] = False
df['BRI_count'] = 0

# defragment frame (for performance)
df = df.copy()

/tmp/ipykernel_838862/4206277511.py:54: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[best_intensity_rapid_intensification_column_name] = False
/tmp/ipykernel_838862/4206277511.py:61: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[forecast_intensity_change_column_name] = np.NaN
/tmp/ipykernel_838862/4206277511.py:68: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) in

In [5]:
# calculate forecast intensities and rapid intensification

# make sure to sort so that we always calculate the forecast_base_intensity first in the inner loop below
sorted_intensity_err_hour_to_column_name_items = sorted(intensity_err_hour_to_column_name.items(), key=lambda x:x[0])

for idx in range(0, len(df)):
    base_time = df.iloc[idx]['ds']
    stmid = df.iloc[idx]['STMID']
    forecast_base_intensity = np.NaN
    forecast_rapid_intensification_any = False
    best_intensity_rapid_intensification_any = False
    for [valid_h, valid_column_name] in sorted_intensity_err_hour_to_column_name_items:
        intensity_err = df.loc[idx][valid_column_name]
        best_intensity = df.loc[idx]['WS']
        if not np.isnan(intensity_err):
            valid_time = base_time + timedelta(hours=valid_h)
            valid_best_intensity_row = df.loc[(df['ds'] == valid_time) & (df['STMID'] == stmid)]
            if not valid_best_intensity_row.empty:
                valid_best_intensity = valid_best_intensity_row['WS'].iloc[0]
                if not np.isnan(valid_best_intensity):
                    # calculate forecast intensity from error and best_intensity
                    forecast_intensity = valid_best_intensity + intensity_err

                    # set the base intensity for this forecast
                    if valid_h == 0:
                        forecast_base_intensity = forecast_intensity
                    
                    # set forecast intensity for valid hour
                    forecast_intensity_column_name = forecast_intensity_hour_to_column_name[valid_h]
                    df.at[idx, forecast_intensity_column_name] = forecast_intensity
                    
                    # set best intensity for valid hour
                    best_intensity_column_name = best_intensity_hour_to_column_name[valid_h]
                    df.at[idx, best_intensity_column_name] = valid_best_intensity

                    # calculate forecast rapid intensification for (selected) forecast hours
                    forecast_rapid_intensification_column_name = forecast_rapid_intensification_hour_to_column_name[valid_h]
                    forecast_rapid_intensification = is_rapid_intensification(valid_h, forecast_base_intensity, forecast_intensity)
                    df.at[idx, forecast_rapid_intensification_column_name] = forecast_rapid_intensification
                    if forecast_rapid_intensification:
                        df.at[idx, 'FRI_count'] = df.at[idx, 'FRI_count'] + 1
                        forecast_rapid_intensification_any = True
                    
                    # calculate best intensity rapid intensification for (selected) forecast hours
                    best_intensity_rapid_intensification_column_name = best_intensity_rapid_intensification_hour_to_column_name[valid_h]
                    best_intensity_rapid_intensification = is_rapid_intensification(valid_h, best_intensity, valid_best_intensity)
                    df.at[idx, best_intensity_rapid_intensification_column_name] = best_intensity_rapid_intensification
                    if best_intensity_rapid_intensification:
                        df.at[idx, 'BRI_count'] = df.at[idx, 'BRI_count'] + 1
                        best_intensity_rapid_intensification_any = True
                    
                    # calculate the change in forecast intensity at valid hour
                    forecast_intensity_change = forecast_intensity - forecast_base_intensity
                    forecast_intensity_change_column_name = forecast_intensity_change_hour_to_column_name[valid_h]
                    df.at[idx, forecast_intensity_change_column_name] = forecast_intensity_change
                    
                    # calculate the change in best intensity at valid hour
                    best_intensity_change = valid_best_intensity - best_intensity
                    best_intensity_change_column_name = best_intensity_change_hour_to_column_name[valid_h]
                    df.at[idx, best_intensity_change_column_name] = best_intensity_change
                else:
                    print(f"Warning: best_intensity data not available for {valid_h} from forecast index {idx}...")
            else:
                # raise warning
                # miss best_intensity calculations if there is no best track 'ws' data here?
                print(f"Warning: no forecast row for {valid_h} starting from forecast index {idx}...")
    df.at[idx, 'FRI'] = forecast_rapid_intensification_any
    df.at[idx, 'BRI'] = best_intensity_rapid_intensification_any


In [6]:
list(df.keys())

['ds',
 'STMID',
 'F012',
 'F024',
 'F036',
 'F048',
 'F060',
 'F072',
 'F096',
 'F120',
 'F144',
 'F168',
 'Lat',
 'Lon',
 'WS',
 '000hT01',
 '012hT01',
 '024hT01',
 '036hT01',
 '048hT01',
 '060hT01',
 '072hT01',
 '096hT01',
 '120hT01',
 '144hT01',
 '168hT01',
 '000hI01',
 '012hI01',
 '024hI01',
 '036hI01',
 '048hI01',
 '060hI01',
 '072hI01',
 '096hI01',
 '120hI01',
 '144hI01',
 '168hI01',
 '0hFI',
 '0hBI',
 '0hFRI',
 '0hBRI',
 '0hDFI',
 '0hDBI',
 '12hFI',
 '12hBI',
 '12hFRI',
 '12hBRI',
 '12hDFI',
 '12hDBI',
 '24hFI',
 '24hBI',
 '24hFRI',
 '24hBRI',
 '24hDFI',
 '24hDBI',
 '36hFI',
 '36hBI',
 '36hFRI',
 '36hBRI',
 '36hDFI',
 '36hDBI',
 '48hFI',
 '48hBI',
 '48hFRI',
 '48hBRI',
 '48hDFI',
 '48hDBI',
 '60hFI',
 '60hBI',
 '60hFRI',
 '60hBRI',
 '60hDFI',
 '60hDBI',
 '72hFI',
 '72hBI',
 '72hFRI',
 '72hBRI',
 '72hDFI',
 '72hDBI',
 '96hFI',
 '96hBI',
 '96hFRI',
 '96hBRI',
 '96hDFI',
 '96hDBI',
 '120hFI',
 '120hBI',
 '120hFRI',
 '120hBRI',
 '120hDFI',
 '120hDBI',
 '144hFI',
 '144hBI',
 '144hFR

In [7]:
# save computation
#df.to_parquet()
file_name_parquet = 'NHC_OFCL_' + header['basin'].replace(' ', '_') + '_' + error_category + '.parquet'
df.to_parquet(file_name_parquet)
print(file_name_parquet)

NHC_OFCL_ATLANTIC_1989-2022_TI.parquet


In [8]:
###### CHECKPOINT

In [9]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

file_name_parquet = 'NHC_OFCL_ATLANTIC_1989-2022_TI.parquet'

In [10]:
df_ri = pd.read_parquet(file_name_parquet)

In [11]:
# repeated as above (without df modifications) so it works with checkpointing
# create mappings and create new columns for calculations and analysis

column_names = list(df.keys())
# examples of column names with 48 hours in parenthesis
# (48hI01) intensity valid forecast hours
intensity_err_column_name_to_hour = {}
intensity_err_hour_to_column_name = {}
# (48hFI) forecast intensity for valid hours of the forecast (missing so recalculate from data)
forecast_intensity_column_name_to_hour = {}
forecast_intensity_hour_to_column_name = {}
# (48hBI) best intensity for valid hours of the forecast
best_intensity_column_name_to_hour = {}
best_intensity_hour_to_column_name = {}
# (48hFRI) forecast rapid intensification
forecast_rapid_intensification_column_name_to_hour = {}
forecast_rapid_intensification_hour_to_column_name = {}
# (48hBRI) best intensity rapid intensification
best_intensity_rapid_intensification_column_name_to_hour = {}
best_intensity_rapid_intensification_hour_to_column_name = {}
# (48hDFI) changes in forecast intensity from base time to valid hour
forecast_intensity_change_column_name_to_hour = {}
forecast_intensity_change_hour_to_column_name = {}
# (48hDBI) changes in best intensity from base time to valid hour
best_intensity_change_column_name_to_hour = {}
best_intensity_change_hour_to_column_name = {}

for column_name in column_names:
    if 'hI' in column_name:
        # create mappings from intensity column names to hour
        h = int(column_name.split('hI')[0])
        intensity_err_column_name_to_hour[column_name] = h
        intensity_err_hour_to_column_name[h] = column_name
        
        # create extra columns for forecast intensity for each forecast hour
        forecast_intensity_column_name = f'{h}hFI'
        forecast_intensity_column_name_to_hour[forecast_intensity_column_name] = h
        forecast_intensity_hour_to_column_name[h] = forecast_intensity_column_name
        
        # create extra columns for best intensity for each forecast hour
        best_intensity_column_name = f'{h}hBI'
        best_intensity_column_name_to_hour[best_intensity_column_name] = h
        best_intensity_hour_to_column_name[h] = best_intensity_column_name

        # create extra columns for forecast RI categorization for each forecast hour
        forecast_rapid_intensification_column_name = f'{h}hFRI'
        forecast_rapid_intensification_column_name_to_hour[forecast_rapid_intensification_column_name] = h
        forecast_rapid_intensification_hour_to_column_name[h] = forecast_rapid_intensification_column_name
        
        # create extra columns for best intensity RI categorization for each forecast hour
        best_intensity_rapid_intensification_column_name = f'{h}hBRI'
        best_intensity_rapid_intensification_column_name_to_hour[best_intensity_rapid_intensification_column_name] = h
        best_intensity_rapid_intensification_hour_to_column_name[h] = best_intensity_rapid_intensification_column_name
        
        # create extra columns for changes in forecast intensity for each forecast hour
        # (forecast intensity at valid hour - forecast intensity at base hour)
        forecast_intensity_change_column_name = f'{h}hDFI'
        forecast_intensity_change_column_name_to_hour[forecast_intensity_change_column_name] = h
        forecast_intensity_change_hour_to_column_name[h] = forecast_intensity_change_column_name
        
        # create extra columns for changes in (best) intensity for each forecast hour
        # (best intensity at valid hour (from other row) - best intensity at base hour)
        best_intensity_change_column_name = f'{h}hDBI'
        best_intensity_change_column_name_to_hour[best_intensity_change_column_name] = h
        best_intensity_change_hour_to_column_name[h] = best_intensity_change_column_name

In [ ]:



######## STATS




In [90]:
### statistics for forecasted rapid intensification (FRI)
### (forecast RI)

print("Statistics for storms with forecast intensity rapid intensification")
print("===================================================================")

fri_num_events = df_ri.loc[df_ri['FRI'] == True]['FRI_count'].sum()
fri_forecast_count = len(df_ri.loc[df_ri['FRI'] == True])
print(f"Number of forecast rapid intensification events on a valid hour: {fri_num_events}")
print(f"Number of forecasts with a forecast rapid intensification event on a valid hour: {fri_forecast_count}")
fri_storm_ids = set()
for idx in df_ri.loc[df_ri['FRI'] == True].index:
    fri_storm_ids.add(df_ri.at[idx,'STMID'])
print(f"{len(fri_storm_ids)} Storms with rapid intensification in forecast:")
print(fri_storm_ids)
print("")

fri_errors = {}
print("Number of forecast rapid intensification events for each forecast valid hour:")
for column_name in forecast_rapid_intensification_column_name_to_hour.keys():
    fri_rows = df_ri.loc[df_ri[column_name] == True]
    count = len(fri_rows)
    if count > 0:
        print(f"{column_name} : {count}")
    
    valid_h = forecast_rapid_intensification_column_name_to_hour[column_name]
    intensity_err_column_name = intensity_err_hour_to_column_name[valid_h]
    fri_errors[valid_h] = []
    for idx in fri_rows.index:
        prev_fri = fri_errors[valid_h]
        intensity_err = fri_rows.at[idx,intensity_err_column_name]
        prev_fri.append(intensity_err)
        fri_errors[valid_h] = prev_fri

# remove empty keys
fri_errors = dict((k, v) for k, v in fri_errors.items() if v)

print("")
print("Table 1.")
# convert to dataframe and compute some stats
print("Forecast intensity error for forecast rapid intensification (columns are forecast valid hours):")
df_fri_errors = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fri_errors.items() ]))
df_fri_errors.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9])

Statistics for storms with forecast intensity rapid intensification
Number of forecast rapid intensification events on a valid hour: 186
Number of forecasts with a forecast rapid intensification event on a valid hour: 118
65 Storms with rapid intensification in forecast:
{'AL091996', 'AL192020', 'AL152022', 'AL072010', 'AL262020', 'AL051996', 'AL052000', 'AL091993', 'AL142018', 'AL132022', 'AL092017', 'AL062002', 'AL282020', 'AL131998', 'AL112001', 'AL072000', 'AL122021', 'AL131996', 'AL052019', 'AL101996', 'AL132020', 'AL092022', 'AL042005', 'AL061996', 'AL071993', 'AL042009', 'AL031998', 'AL182005', 'AL252005', 'AL062006', 'AL112017', 'AL051993', 'AL041999', 'AL131999', 'AL312020', 'AL081996', 'AL172008', 'AL292020', 'AL112000', 'AL061991', 'AL092021', 'AL182021', 'AL041996', 'AL102000', 'AL191995', 'AL031996', 'AL051997', 'AL132002', 'AL152001', 'AL041992', 'AL101995', 'AL021998', 'AL071992', 'AL052001', 'AL142017', 'AL132010', 'AL112010', 'AL132000', 'AL202020', 'AL072008', 'AL0119

,12,24,36,48
count,86.000000,67.000000,21.000000,12.000000
mean,-0.883721,-1.641791,-6.904762,-1.250000
std,12.503570,18.553527,14.359334,14.000812
min,-60.000000,-40.000000,-35.000000,-30.000000
10%,-15.000000,-25.000000,-30.000000,-23.000000
25%,-5.000000,-12.500000,-15.000000,-5.000000
50%,0.000000,0.000000,-5.000000,0.000000
75%,5.000000,7.500000,0.000000,6.250000
90%,12.500000,20.000000,10.000000,14.500000
max,30.000000,45.000000,15.000000,15.000000


In [82]:
### statistics for best intensity rapid intensification (BRI)
## ("observed" RI)

print("Statistics for storms with best intensity rapid intensification:")
print("================================================================")

bri_num_events = df_ri.loc[df_ri['BRI'] == True]['BRI_count'].sum()
bri_forecast_count = len(df_ri.loc[df_ri['BRI'] == True])
print(f"Number of best intensity rapid intensification events on valid hours: {bri_num_events}")
print(f"Number of forecasts with a best intensity rapid intensification event on a valid hour: {bri_forecast_count}")
bri_storm_ids = set()
for idx in df_ri.loc[df_ri['BRI'] == True].index:
    bri_storm_ids.add(df_ri.at[idx,'STMID'])
print(f"{len(bri_storm_ids)} Storms with best intensity rapid intensification:")
print("")
print(bri_storm_ids)
print("")

bri_errors = {}
print("Number of best intensity rapid intensification events for each forecast valid hour:")
for column_name in best_intensity_rapid_intensification_column_name_to_hour.keys():
    bri_rows = df_ri.loc[df_ri[column_name] == True]
    count = len(bri_rows)
    if count > 0:
        print(f"{column_name} : {count}")
    
    valid_h = best_intensity_rapid_intensification_column_name_to_hour[column_name]
    intensity_err_column_name = intensity_err_hour_to_column_name[valid_h]
    bri_errors[valid_h] = []
    for idx in bri_rows.index:
        prev_bri = bri_errors[valid_h]
        intensity_err = bri_rows.at[idx,intensity_err_column_name]
        prev_bri.append(intensity_err)
        bri_errors[valid_h] = prev_bri

# remove empty keys
bri_errors = dict((k, v) for k, v in bri_errors.items() if v)

print("")
print("Table 2.")
# convert to dataframe and compute some stats
print("Forecast intensity error for best intensity rapid intensification (columns are forecast valid hours):")
df_bri_errors = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in bri_errors.items() ]))
df_bri_errors.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9])

Statistics for storms with best intensity rapid intensification:
Number of best intensity rapid intensification events on valid hours: 1679
Number of forecasts with a best intensity rapid intensification event on a valid hour: 945
181 Storms with best intensity rapid intensification:

{'AL091996', 'AL052005', 'AL042008', 'AL161990', 'AL072010', 'AL262020', 'AL051996', 'AL021996', 'AL052022', 'AL182010', 'AL082002', 'AL142016', 'AL142018', 'AL092013', 'AL091998', 'AL282020', 'AL142010', 'AL172010', 'AL122021', 'AL032008', 'AL092006', 'AL101996', 'AL102019', 'AL092004', 'AL082006', 'AL042009', 'AL182005', 'AL092007', 'AL041999', 'AL062007', 'AL172008', 'AL102001', 'AL122005', 'AL072009', 'AL162016', 'AL041996', 'AL132019', 'AL182021', 'AL112004', 'AL072021', 'AL082022', 'AL021995', 'AL132010', 'AL132000', 'AL011995', 'AL051992', 'AL082014', 'AL062015', 'AL032001', 'AL122004', 'AL071997', 'AL062018', 'AL032005', 'AL032000', 'AL082020', 'AL032018', 'AL122010', 'AL091995', 'AL151995', 'AL11

,12,24,36,48
count,459.000000,565.000000,340.000000,315.000000
mean,-17.159041,-23.283186,-32.411765,-36.603175
std,9.422331,11.262085,14.097464,15.243814
min,-60.000000,-70.000000,-80.000000,-95.000000
10%,-30.000000,-35.000000,-50.000000,-55.000000
25%,-20.000000,-30.000000,-40.000000,-45.000000
50%,-15.000000,-20.000000,-30.000000,-35.000000
75%,-10.000000,-15.000000,-25.000000,-25.000000
90%,-5.000000,-10.000000,-15.000000,-20.000000
max,10.000000,5.000000,5.000000,5.000000


In [91]:
### Basic statistics for number of storms and RI storms
print("Basic statistics for all storms")
print("===============================")

all_storm_ids = set(df_ri['STMID'].to_list())
num_storms = len(all_storm_ids)
pct_bri = 100.0 * len(bri_storm_ids) / num_storms
pct_no_bri = 100.0 - pct_bri
pct_fri = 100.0 * len(fri_storm_ids) / num_storms
pct_no_fri = 100.0 - pct_fri
print(f"Total storms: {num_storms}")
print("")
print(f"% of all storms with best intensity rapid intensification: {pct_bri:2.2f} %")
print(f"% of all storms with no best intensity rapid intensification: {pct_no_bri:2.2f} %")
print("")
print(f"% of all storms with forecast rapid intensification: {pct_fri:2.2f} %")
print(f"% of all storms with no forecast rapid intensification: {pct_no_fri:2.2f} %")

Basic statistics for all storms
Total storms: 545

% of all storms with best intensity rapid intensification: 33.21 %
% of all storms with no best intensity rapid intensification: 66.79 %

% of all storms with forecast rapid intensification: 11.93 %
% of all storms with no forecast rapid intensification: 88.07 %


In [84]:
### Statistics for storms with no best intensity rapid intensification
## ("observed" NO RI)

no_bri_storm_ids = all_storm_ids.difference(bri_storm_ids)
# the number of valid forecast hour columns (including base time)
num_valid_hour_columns = len(best_intensity_rapid_intensification_hour_to_column_name)

df_no_bri = df_ri.loc[df_ri['BRI'] == False]
no_bri_num_events = (len(df_no_bri) * (num_valid_hour_columns - 1))

# subtract one since the base time can never have rapid intensification
no_bri_forecast_count = len(df_no_bri)

print("Statistics for storms with no best intensity rapid intensification:")
print("===================================================================")
print(f"Number of best intensity no rapid intensification events on valid hours: {no_bri_num_events}")
print(f"Number of forecasts with no best intensity rapid intensification event on a valid hour: {no_bri_forecast_count}")
print(f"{len(no_bri_storm_ids)} Storms with no best intensity rapid intensification:")
print("")
print(no_bri_storm_ids)
print("")

no_bri_errors = {}
for [valid_h, column_name] in intensity_err_hour_to_column_name.items():
    no_bri_errors[valid_h] = []
    for idx in df_no_bri.index:
        prev_no_bri = no_bri_errors[valid_h]
        intensity_err = df_no_bri.at[idx, column_name]
        prev_no_bri.append(intensity_err)
        no_bri_errors[valid_h] = prev_no_bri

# remove empty keys
no_bri_errors = dict((k, v) for k, v in no_bri_errors.items() if v)

print("")
print("Table 3.")
# convert to dataframe and compute some stats
print("Forecast intensity error for best intensity no rapid intensification (columns are forecast valid hours):")
df_no_bri_errors = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in no_bri_errors.items() ]))
df_no_bri_errors.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9])

Statistics for storms with no best intensity rapid intensification:
Number of best intensity no rapid intensification events on valid hours: 113670
Number of forecasts with no best intensity rapid intensification event on a valid hour: 11367
364 Storms with no best intensity rapid intensification:

{'AL122008', 'AL082009', 'AL081994', 'AL072003', 'AL092009', 'AL082000', 'AL021993', 'AL062011', 'AL132008', 'AL032007', 'AL232020', 'AL032019', 'AL111990', 'AL102005', 'AL052004', 'AL092003', 'AL012018', 'AL142021', 'AL202021', 'AL172017', 'AL012022', 'AL192017', 'AL092012', 'AL222005', 'AL011998', 'AL182003', 'AL282005', 'AL132021', 'AL041993', 'AL032011', 'AL102007', 'AL071999', 'AL022013', 'AL102004', 'AL062019', 'AL101993', 'AL112018', 'AL132011', 'AL102021', 'AL022020', 'AL192011', 'AL021992', 'AL012014', 'AL142007', 'AL152022', 'AL102011', 'AL052010', 'AL071989', 'AL101999', 'AL042010', 'AL042017', 'AL031992', 'AL072019', 'AL122012', 'AL071991', 'AL082010', 'AL012013', 'AL182017', 'AL

,0,12,24,36,48,60,72,96,120,144,168
count,9923.000000,8940.000000,7935.000000,7000.000000,6155.000000,711.000000,4792.000000,2439.000000,1880.000000,0.0,0.0
mean,-0.746246,0.928971,1.975425,2.147857,2.086109,1.188467,1.387730,-0.194752,-1.348404,NaN,NaN
std,4.040700,7.709195,10.983638,13.524659,15.970980,13.401742,20.704668,21.711836,23.912280,NaN,NaN
min,-35.000000,-30.000000,-45.000000,-50.000000,-65.000000,-50.000000,-90.000000,-90.000000,-100.000000,NaN,NaN
10%,-5.000000,-10.000000,-10.000000,-15.000000,-20.000000,-15.000000,-25.000000,-25.000000,-30.000000,NaN,NaN
25%,0.000000,-5.000000,-5.000000,-5.000000,-10.000000,-5.000000,-10.000000,-10.000000,-15.000000,NaN,NaN
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
75%,0.000000,5.000000,10.000000,10.000000,10.000000,10.000000,15.000000,15.000000,15.000000,NaN,NaN
90%,5.000000,10.000000,15.000000,20.000000,20.000000,15.000000,25.000000,25.000000,25.000000,NaN,NaN
max,30.000000,50.000000,75.000000,85.000000,95.000000,40.000000,85.000000,90.000000,90.000000,NaN,NaN


In [89]:
### Statistics for storms with no best intensity rapid intensification but with forecast rapid intensification
## ("observed" NO RI, YES forecast RI)

no_bri_yes_fri_storm_ids = no_bri_storm_ids.intersection(fri_storm_ids)
# the number of valid forecast hour columns (including base time)
num_valid_hour_columns = len(best_intensity_rapid_intensification_hour_to_column_name)

df_no_bri_yes_fri = df_no_bri_yes_fri = df_ri.loc[(df_ri['BRI'] == False) & (df_ri['FRI'] == True)]
no_bri_yes_fri_num_events = df_no_bri_yes_fri['FRI_count'].sum()
no_bri_yes_fri_forecast_count = len(df_no_bri_yes_fri)

# subtract one since the base time can never have rapid intensification
no_bri_yes_fri_forecast_count = len(df_no_bri_yes_fri)

print("Statistics for storms with no best intensity rapid intensification but with forecast RI:")
print("========================================================================================")
print(f"Number of forecast rapid intensification events on valid hours: {no_bri_yes_fri_num_events}")
print(f"Number of forecasts with rapid intensification event on a valid hour: {no_bri_yes_fri_forecast_count}")
print(f"{len(no_bri_yes_fri_storm_ids)} Storms with forecast rapid intensification:")
print("")
print(no_bri_yes_fri_storm_ids)
print("")

no_bri_yes_fri_errors = {}
print("Number of best intensity rapid intensification events for each forecast valid hour:")
for column_name in forecast_rapid_intensification_column_name_to_hour.keys():
    no_bri_yes_fri_rows = df_no_bri_yes_fri.loc[df_no_bri_yes_fri[column_name] == True]
    count = len(no_bri_yes_fri_rows)
    if count > 0:
        print(f"{column_name} : {count}")
    
    valid_h = forecast_rapid_intensification_column_name_to_hour[column_name]
    intensity_err_column_name = intensity_err_hour_to_column_name[valid_h]
    no_bri_yes_fri_errors[valid_h] = []
    for idx in no_bri_yes_fri_rows.index:
        prev_no_bri_yes_fri = no_bri_yes_fri_errors[valid_h]
        intensity_err = no_bri_yes_fri_rows.at[idx,intensity_err_column_name]
        prev_no_bri_yes_fri.append(intensity_err)
        no_bri_yes_fri_errors[valid_h] = prev_no_bri_yes_fri

# remove empty keys
no_bri_yes_fri_errors = dict((k, v) for k, v in no_bri_yes_fri_errors.items() if v)

print("")
print("Table 4.")
# convert to dataframe and compute some stats
print("Forecast intensity error for best intensity no rapid intensification but with forecast RI\n(columns are forecast valid hours):")
df_no_bri_yes_fri_errors = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in no_bri_yes_fri_errors.items() ]))
df_no_bri_yes_fri_errors.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9])

Statistics for storms with no best intensity rapid intensification but with forecast RI:
Number of forecast rapid intensification events on valid hours: 62
Number of forecasts with rapid intensification event on a valid hour: 49
10 Storms with forecast rapid intensification:

{'AL051993', 'AL071992', 'AL152022', 'AL011994', 'AL052000', 'AL061996', 'AL111996', 'AL132022', 'AL052001', 'AL062002'}

Number of best intensity rapid intensification events for each forecast valid hour:
12hFRI : 42
24hFRI : 18
48hFRI : 2

Table 4.
Forecast intensity error for best intensity no rapid intensification but with forecast RI
(columns are forecast valid hours):


,12,24,48
count,42.000000,18.000000,2.0
mean,6.428571,16.111111,15.0
std,7.265498,14.095844,0.0
min,-5.000000,-5.000000,15.0
10%,0.000000,5.000000,15.0
25%,1.250000,5.000000,15.0
50%,5.000000,15.000000,15.0
75%,10.000000,20.000000,15.0
90%,15.000000,38.000000,15.0
max,30.000000,45.000000,15.0


In [86]:
# Note: storms forecast RI that also have best intensity RI (not necessarily at the correct valid time)

no_bri_storm_ids = all_storm_ids.difference(bri_storm_ids)
no_fri_storm_ids = all_storm_ids.difference(fri_storm_ids)

# naive validation since forecast may have gotten RI for the wrong time/location/reason
tp = len(fri_storm_ids.intersection(bri_storm_ids))
pct_tp = 100.0 * tp / len(bri_storm_ids)
fn = len(no_fri_storm_ids.intersection(bri_storm_ids))
pct_fn = 100.0 * fn / len(bri_storm_ids)

print("RI Storms:")
print(f"% Storms with best intensity RI that were forecast with RI (TP): {pct_tp:2.2f} % ({tp})")
print(f"% Storms with best intensity RI that were not forecast with RI (FN): {pct_fn:2.2f} % ({fn})")
print("")
print("Non-RI Storms:")
fp = len(fri_storm_ids.intersection(no_bri_storm_ids))
pct_fp = 100.0 * fp / len(no_bri_storm_ids)
tn = len(no_fri_storm_ids.intersection(no_bri_storm_ids))
pct_tn = 100.0 * tn / len(no_bri_storm_ids)
print(f"% Storms with no best intensity RI that were forecast with RI (FP): {pct_fp:2.2f} % ({fp})")
print(f"% Storms with no best intensity RI that were not forecast with RI (TN): {pct_tn:2.2f} % ({tn})")


RI Storms:
% Storms with best intensity RI that were forecast with RI (TP): 30.39 % (55)
% Storms with best intensity RI that were not forecast with RI (FN): 69.61 % (126)

Non-RI Storms:
% Storms with no best intensity RI that were forecast with RI (FP): 2.75 % (10)
% Storms with no best intensity RI that were not forecast with RI (TN): 97.25 % (354)


In [52]:

# Initial thoughts...

# Hypothetical basis for considering probabilities using OFCL intensities alone from forecast advisories:
#   when calculating base rates using the above tables and statistics, check if there is RI in the forecast:
#   if there is RI in the forecast:
#      note that % RI false positives is only ~ 3%: so reference,
#         2.75 % to weight (pct_fp) on Table 4 calculations
#         97.25 % to weight (pct_fp) on Table 1 calculations
#
#   if there is no RI in the forecast:
#      note only ~33% of storms have RI historically (in the last ~30 years): so reference,
#         66.79 % to weight (pct_no_bri) on Table 3 calculations
#         33.21 % to weight (pct_bri) on Table 2 calculations